# Import all necessary library

In [62]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.data import Dataset
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.layers import Input, LSTM, Dense, Bidirectional, Activation, GRU
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import timeseries_dataset_from_array
from tensorflow.keras.callbacks import LambdaCallback
from sklearn import tree
from sklearn.metrics import confusion_matrix
import itertools
import time
import multiprocessing
from pickle import dump
import warnings

warnings.filterwarnings('ignore')


%matplotlib inline

gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
print(gpus)
tf.config.experimental.set_visible_devices(devices=gpus[0], device_type='GPU')
tf.config.experimental.set_memory_growth(device=gpus[0], enable=True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Load all data and make a summary

In [63]:
pwd = os.getcwd()
all_file_names = []
scenarios = ['scenario'+str(i+1) for i in range(5)]
data_dist_table = pd.DataFrame(index=['Hold','Takeoff','Hover','Search','Loiter','Obstacleavoid','Land'])
for scenario in scenarios:
    count_dict = {'Hold':0,'Takeoff':0,'Hover':0,'Search':0,'Loiter':0,'Obstacleavoid':0,'Land':0}
    database_path = pwd[:-6] + "flightData/filtered_data/"+scenario
    database_file_names = []
    for root, dirs, files in os.walk(database_path):
        database_file_names = [os.path.join(root,file) for file in files]
    all_file_names += database_file_names
    for file in database_file_names:
        data = pd.read_csv(file)
        label_count = data['label'].value_counts()
        for indx in label_count.index:
            count_dict[indx] += label_count[indx]
    data_dist_table[scenario] = count_dict.values()
print(data_dist_table)            

               scenario1  scenario2  scenario3  scenario4  scenario5
Hold               13264       3378      64310      15361      33644
Takeoff             1813       1850       1900       1978       1974
Hover               9001       9036       9045       9039       9025
Search                 0      40868     204655     154459     156413
Loiter                 0          0      83451          0      49397
Obstacleavoid          0          0          0       7665       9119
Land                1940       1974       1986       1993       2027


# LSTM Implementation

# Training

In [64]:
%%time

# training

# deep LSTM model
time_steps = [128] #[32, 64, 128]
num_features = [7] #[7,9]
num_class = 7
direction = 2
neuron_sizes = [64] #[64, 128, 256, 512]
lstm_layers = 1

for time_step in time_steps:
    for num_feature in num_features:
        for neuron_size in neuron_sizes:
            # create model
            x_in = Input(shape=(time_step,num_feature),dtype="float32")
            bid_layer1 = Bidirectional(LSTM(neuron_size))(x_in)
            y = Dense(units=num_class, activation='softmax')(bid_layer1)

            model = Model(inputs=x_in, outputs=y)
            optimizer = optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, amsgrad=False)
            loss = losses.categorical_crossentropy
            model.compile(optimizer=optimizer, loss=loss, metrics=['CategoricalAccuracy'])


            # load dataset
            pwd = os.getcwd()
            model_path = pwd+'/models/'

            batch_print_callback = LambdaCallback(on_batch_end=lambda batch,logs: print(batch, logs))

            pwd = os.getcwd()
            database_path = pwd[:-6] + "flightData/filtered_data/scenario5"
            database_file_names = []
            for root, dirs, files in os.walk(database_path):
                database_file_names = [os.path.join(root,file) for file in files]

            scaler = MinMaxScaler()
            encoder = OneHotEncoder()
            train_set = database_file_names[:-1]
            #training loop
            for model_no in range(1,71):
                for i,file in enumerate(train_set):
                    data = pd.read_csv(file)
                    if num_feature == 7:
                        X_train = data.drop(columns=['label', 'time','x','y','pitch_v','roll_v','yaw_v'])
                    else:
                        X_train = data.drop(columns=['label', 'time','pitch_v','roll_v','yaw_v'])

                    X_train = X_train.values
                    Y_train = data['label'].values
                    Y_train = Y_train.reshape((-1,1))
                    if i == 0:
                        X_train = scaler.fit_transform(X_train)
                        Y_train = encoder.fit_transform(Y_train).toarray()
                    else:
                        X_train = scaler.transform(X_train)
                        Y_train = encoder.transform(Y_train).toarray()

                    input_data = X_train[:-time_step,:]
                    targets = Y_train[time_step:,:]
                    dataset = timeseries_dataset_from_array(input_data, 
                                                            targets, 
                                                            sequence_length=time_step,
                                                            batch_size=128)
                    hist = model.fit(dataset,
                                     epochs=10,
                                     verbose=2,
                                     callbacks=None,#[batch_print_callback],
                                     max_queue_size=512,
                                     workers=80,
                                     use_multiprocessing=True)

#                 model_name = 'lstm_direction_' + str(direction)\
#                              + '_layers_' + str(lstm_layers)\
#                              + '_features_' + str(num_feature)\
#                              + '_neurons_' + str(neuron_size)\
#                              + '_timesteps_' + str(time_step)\
#                              + '_epochs_' + str((model_no)*10) + '.h5'
#                 model.save(model_path+model_name) 


Epoch 1/10
425/425 - 11s - loss: 1.2506 - categorical_accuracy: 0.6707
Epoch 2/10
425/425 - 11s - loss: 1.0385 - categorical_accuracy: 0.6497
Epoch 3/10
425/425 - 11s - loss: 0.9830 - categorical_accuracy: 0.6340
Epoch 4/10
425/425 - 10s - loss: 0.9465 - categorical_accuracy: 0.6583
Epoch 5/10
425/425 - 11s - loss: 0.9188 - categorical_accuracy: 0.6712
Epoch 6/10
425/425 - 11s - loss: 1.0689 - categorical_accuracy: 0.6302
Epoch 7/10
425/425 - 10s - loss: 0.9622 - categorical_accuracy: 0.6434
Epoch 8/10
425/425 - 11s - loss: 0.9435 - categorical_accuracy: 0.6445
Epoch 9/10
425/425 - 11s - loss: 0.8896 - categorical_accuracy: 0.6665
Epoch 10/10
425/425 - 11s - loss: 0.8163 - categorical_accuracy: 0.6874
Epoch 1/10
304/304 - 7s - loss: 0.8645 - categorical_accuracy: 0.7038
Epoch 2/10
304/304 - 7s - loss: 1.0259 - categorical_accuracy: 0.7046
Epoch 3/10
304/304 - 8s - loss: 0.9260 - categorical_accuracy: 0.7343
Epoch 4/10
304/304 - 8s - loss: 0.8395 - categorical_accuracy: 0.7097
Epoch 5/1

Epoch 7/10
434/434 - 11s - loss: 0.6969 - categorical_accuracy: 0.7579
Epoch 8/10
434/434 - 11s - loss: 0.6393 - categorical_accuracy: 0.7931
Epoch 9/10
434/434 - 11s - loss: 0.5938 - categorical_accuracy: 0.8040
Epoch 10/10
434/434 - 11s - loss: 0.5934 - categorical_accuracy: 0.8007
Epoch 1/10
425/425 - 11s - loss: 0.6444 - categorical_accuracy: 0.7716
Epoch 2/10
425/425 - 11s - loss: 0.6262 - categorical_accuracy: 0.7549
Epoch 3/10
425/425 - 11s - loss: 0.5528 - categorical_accuracy: 0.7912
Epoch 4/10
425/425 - 11s - loss: 0.6219 - categorical_accuracy: 0.7601
Epoch 5/10
425/425 - 11s - loss: 0.6079 - categorical_accuracy: 0.7582
Epoch 6/10
425/425 - 11s - loss: 0.5477 - categorical_accuracy: 0.8079
Epoch 7/10
425/425 - 10s - loss: 0.5348 - categorical_accuracy: 0.8004
Epoch 8/10
425/425 - 10s - loss: 0.6204 - categorical_accuracy: 0.8163
Epoch 9/10
425/425 - 11s - loss: 0.5659 - categorical_accuracy: 0.7853
Epoch 10/10
425/425 - 11s - loss: 0.6511 - categorical_accuracy: 0.7615
Epoc

Epoch 3/10
434/434 - 11s - loss: 0.6438 - categorical_accuracy: 0.7891
Epoch 4/10
434/434 - 11s - loss: 0.7253 - categorical_accuracy: 0.7618
Epoch 5/10
434/434 - 11s - loss: 0.6908 - categorical_accuracy: 0.7682
Epoch 6/10
434/434 - 11s - loss: 0.6744 - categorical_accuracy: 0.7769
Epoch 7/10
434/434 - 11s - loss: 0.6414 - categorical_accuracy: 0.7860
Epoch 8/10
434/434 - 11s - loss: 0.6067 - categorical_accuracy: 0.8171
Epoch 9/10
434/434 - 11s - loss: 0.6434 - categorical_accuracy: 0.7834
Epoch 10/10
434/434 - 11s - loss: 0.6367 - categorical_accuracy: 0.7869
Epoch 1/10
425/425 - 11s - loss: 0.5265 - categorical_accuracy: 0.8017
Epoch 2/10
425/425 - 11s - loss: 0.6026 - categorical_accuracy: 0.7792
Epoch 3/10
425/425 - 11s - loss: 0.5353 - categorical_accuracy: 0.7891
Epoch 4/10
425/425 - 11s - loss: 0.4948 - categorical_accuracy: 0.7984
Epoch 5/10
425/425 - 11s - loss: 0.4721 - categorical_accuracy: 0.7699
Epoch 6/10
425/425 - 11s - loss: 0.4873 - categorical_accuracy: 0.7761
Epoch

Epoch 9/10
466/466 - 12s - loss: 0.4995 - categorical_accuracy: 0.8439
Epoch 10/10
466/466 - 11s - loss: 0.4232 - categorical_accuracy: 0.8546
Epoch 1/10
434/434 - 11s - loss: 0.5370 - categorical_accuracy: 0.7888
Epoch 2/10
434/434 - 11s - loss: 0.4955 - categorical_accuracy: 0.8020
Epoch 3/10
434/434 - 11s - loss: 0.5899 - categorical_accuracy: 0.8071
Epoch 4/10
434/434 - 11s - loss: 0.6333 - categorical_accuracy: 0.8125
Epoch 5/10
434/434 - 11s - loss: 0.5585 - categorical_accuracy: 0.7931
Epoch 6/10
434/434 - 11s - loss: 0.7863 - categorical_accuracy: 0.7599
Epoch 7/10
434/434 - 11s - loss: 0.5809 - categorical_accuracy: 0.8018
Epoch 8/10
434/434 - 11s - loss: 0.7422 - categorical_accuracy: 0.7938
Epoch 9/10
434/434 - 11s - loss: 0.5817 - categorical_accuracy: 0.7834
Epoch 10/10
434/434 - 11s - loss: 0.6433 - categorical_accuracy: 0.7453
Epoch 1/10
425/425 - 11s - loss: 0.5266 - categorical_accuracy: 0.8227
Epoch 2/10
425/425 - 11s - loss: 0.5770 - categorical_accuracy: 0.7980
Epoc

Epoch 5/10
466/466 - 12s - loss: 0.4211 - categorical_accuracy: 0.8299
Epoch 6/10
466/466 - 12s - loss: 0.4337 - categorical_accuracy: 0.8157
Epoch 7/10
466/466 - 12s - loss: 0.4123 - categorical_accuracy: 0.8503
Epoch 8/10
466/466 - 12s - loss: 0.4065 - categorical_accuracy: 0.8524
Epoch 9/10
466/466 - 12s - loss: 0.4594 - categorical_accuracy: 0.8407
Epoch 10/10
466/466 - 12s - loss: 0.4400 - categorical_accuracy: 0.8378
Epoch 1/10
434/434 - 11s - loss: 0.4724 - categorical_accuracy: 0.8187
Epoch 2/10
434/434 - 11s - loss: 0.4499 - categorical_accuracy: 0.8240
Epoch 3/10
434/434 - 11s - loss: 0.4252 - categorical_accuracy: 0.8282
Epoch 4/10
434/434 - 11s - loss: 0.5579 - categorical_accuracy: 0.8189
Epoch 5/10
434/434 - 11s - loss: 0.5048 - categorical_accuracy: 0.8088
Epoch 6/10
434/434 - 11s - loss: 0.4507 - categorical_accuracy: 0.8134
Epoch 7/10
434/434 - 11s - loss: 0.4669 - categorical_accuracy: 0.7995
Epoch 8/10
434/434 - 11s - loss: 0.4580 - categorical_accuracy: 0.8079
Epoch

...

Epoch 9/10
466/466 - 12s - loss: 0.0879 - categorical_accuracy: 0.9733
Epoch 10/10
466/466 - 12s - loss: 0.0754 - categorical_accuracy: 0.9795
Epoch 1/10
434/434 - 11s - loss: 0.1471 - categorical_accuracy: 0.9567
Epoch 2/10
434/434 - 11s - loss: 0.1451 - categorical_accuracy: 0.9563
Epoch 3/10
434/434 - 11s - loss: 0.1496 - categorical_accuracy: 0.9541
Epoch 4/10
434/434 - 11s - loss: 0.1254 - categorical_accuracy: 0.9596
Epoch 5/10
434/434 - 11s - loss: 0.1130 - categorical_accuracy: 0.9622
Epoch 6/10
434/434 - 11s - loss: 0.1171 - categorical_accuracy: 0.9624
Epoch 7/10
434/434 - 11s - loss: 0.1154 - categorical_accuracy: 0.9573
Epoch 8/10
434/434 - 11s - loss: 0.1897 - categorical_accuracy: 0.9458
Epoch 9/10
434/434 - 11s - loss: 0.3218 - categorical_accuracy: 0.9276
Epoch 10/10
434/434 - 11s - loss: 0.3892 - categorical_accuracy: 0.8943
Epoch 1/10
425/425 - 11s - loss: 0.2661 - categorical_accuracy: 0.9175
Epoch 2/10
425/425 - 11s - loss: 0.2085 - categorical_accuracy: 0.9077
Epoc

Epoch 5/10
466/466 - 12s - loss: 0.0682 - categorical_accuracy: 0.9806
Epoch 6/10
466/466 - 12s - loss: 0.0694 - categorical_accuracy: 0.9820
Epoch 7/10
466/466 - 12s - loss: 0.0696 - categorical_accuracy: 0.9815
Epoch 8/10
466/466 - 12s - loss: 0.0757 - categorical_accuracy: 0.9768
Epoch 9/10
466/466 - 12s - loss: 0.0669 - categorical_accuracy: 0.9815
Epoch 10/10
466/466 - 12s - loss: 0.0674 - categorical_accuracy: 0.9817
Epoch 1/10
434/434 - 11s - loss: 0.1501 - categorical_accuracy: 0.9599
Epoch 2/10
434/434 - 11s - loss: 0.1474 - categorical_accuracy: 0.9557
Epoch 3/10
434/434 - 11s - loss: 0.1441 - categorical_accuracy: 0.9517
Epoch 4/10
434/434 - 11s - loss: 0.1301 - categorical_accuracy: 0.9585
Epoch 5/10
434/434 - 11s - loss: 0.1222 - categorical_accuracy: 0.9623
Epoch 6/10
434/434 - 11s - loss: 0.1257 - categorical_accuracy: 0.9602
Epoch 7/10
434/434 - 11s - loss: 0.1272 - categorical_accuracy: 0.9584
Epoch 8/10
434/434 - 11s - loss: 0.1193 - categorical_accuracy: 0.9621
Epoch

Epoch 1/10
434/434 - 11s - loss: 0.1642 - categorical_accuracy: 0.9447
Epoch 2/10
434/434 - 11s - loss: 0.2019 - categorical_accuracy: 0.9332
Epoch 3/10
434/434 - 11s - loss: 0.2085 - categorical_accuracy: 0.9389
Epoch 4/10
434/434 - 11s - loss: 0.1606 - categorical_accuracy: 0.9487
Epoch 5/10
434/434 - 11s - loss: 0.1424 - categorical_accuracy: 0.9517
Epoch 6/10
434/434 - 11s - loss: 0.1435 - categorical_accuracy: 0.9526
Epoch 7/10
434/434 - 11s - loss: 0.1336 - categorical_accuracy: 0.9541
Epoch 8/10
434/434 - 11s - loss: 0.1257 - categorical_accuracy: 0.9554
Epoch 9/10
434/434 - 11s - loss: 0.1247 - categorical_accuracy: 0.9577
Epoch 10/10
434/434 - 11s - loss: 0.1234 - categorical_accuracy: 0.9565
Epoch 1/10
425/425 - 11s - loss: 0.3779 - categorical_accuracy: 0.9075
Epoch 2/10
425/425 - 11s - loss: 0.8625 - categorical_accuracy: 0.8508
Epoch 3/10
425/425 - 11s - loss: 1.0316 - categorical_accuracy: 0.8376
Epoch 4/10
425/425 - 11s - loss: 0.8320 - categorical_accuracy: 0.8157
Epoch

Epoch 7/10
466/466 - 12s - loss: 0.4001 - categorical_accuracy: 0.8831
Epoch 8/10
466/466 - 12s - loss: 0.3608 - categorical_accuracy: 0.8915
Epoch 9/10
466/466 - 12s - loss: 0.3316 - categorical_accuracy: 0.8986
Epoch 10/10
466/466 - 12s - loss: 0.3041 - categorical_accuracy: 0.9028
Epoch 1/10
434/434 - 11s - loss: 0.3507 - categorical_accuracy: 0.8824
Epoch 2/10
434/434 - 11s - loss: 0.3338 - categorical_accuracy: 0.8928
Epoch 3/10
434/434 - 11s - loss: 0.3115 - categorical_accuracy: 0.8929
Epoch 4/10
434/434 - 11s - loss: 0.4206 - categorical_accuracy: 0.8704
Epoch 5/10
434/434 - 11s - loss: 0.3569 - categorical_accuracy: 0.8934
Epoch 6/10
434/434 - 11s - loss: 0.7857 - categorical_accuracy: 0.8154
Epoch 7/10
434/434 - 11s - loss: 0.5338 - categorical_accuracy: 0.8417
Epoch 8/10
434/434 - 11s - loss: 0.4868 - categorical_accuracy: 0.8614
Epoch 9/10
434/434 - 11s - loss: 0.3807 - categorical_accuracy: 0.8782
Epoch 10/10
434/434 - 11s - loss: 0.3293 - categorical_accuracy: 0.8894
Epoc

Epoch 3/10
466/466 - 12s - loss: 0.2073 - categorical_accuracy: 0.9168
Epoch 4/10
466/466 - 12s - loss: 0.3200 - categorical_accuracy: 0.9042
Epoch 5/10
466/466 - 12s - loss: 0.2004 - categorical_accuracy: 0.9277
Epoch 6/10
466/466 - 12s - loss: 0.1945 - categorical_accuracy: 0.9292
Epoch 7/10
466/466 - 12s - loss: 0.1901 - categorical_accuracy: 0.9307
Epoch 8/10
466/466 - 12s - loss: 0.1854 - categorical_accuracy: 0.9320
Epoch 9/10
466/466 - 12s - loss: 0.1834 - categorical_accuracy: 0.9326
Epoch 10/10
466/466 - 12s - loss: 0.1801 - categorical_accuracy: 0.9333
Epoch 1/10
434/434 - 11s - loss: 0.2569 - categorical_accuracy: 0.9083
Epoch 2/10
434/434 - 11s - loss: 0.2421 - categorical_accuracy: 0.9106
Epoch 3/10
434/434 - 11s - loss: 0.2302 - categorical_accuracy: 0.9132
Epoch 4/10
434/434 - 11s - loss: 0.2255 - categorical_accuracy: 0.9147
Epoch 5/10
434/434 - 11s - loss: 0.2245 - categorical_accuracy: 0.9123
Epoch 6/10
434/434 - 11s - loss: 0.2151 - categorical_accuracy: 0.9183
Epoch

# Validation

In [66]:
%%time
# validation
val_file = database_file_names[-1]
data = pd.read_csv(val_file)
X_test = data.drop(columns=['label', 'time','x','y','pitch_v','roll_v','yaw_v'])
X_test = X_test.values
Y_test = data['label'].values
Y_test = Y_test.reshape((-1,1))

X_test = scaler.transform(X_test)
Y_test = encoder.transform(Y_test).toarray()

input_data = X_test[:-time_step,:]
targets = Y_test[time_step:,:]
dataset = timeseries_dataset_from_array(input_data, 
                                        targets, 
                                        sequence_length=time_step,
                                        batch_size=128)
hist = model.evaluate(dataset,
                      verbose=1,
                      callbacks=None,#[batch_print_callback],
                      max_queue_size=512,
                      workers=80,
                      use_multiprocessing=True,
                      return_dict=True)

print(hist['categorical_accuracy'])
    
    


407/407 [==============================] - 6s 14ms/step - loss: 0.4463 - categorical_accuracy: 0.8422
0.8421608209609985
CPU times: user 24.6 s, sys: 27.4 s, total: 52.1 s
Wall time: 5.85 s


# Testing

In [67]:
%%time
# testing
pwd = os.getcwd()

test_result = {}

for scenario_num in range(1,6):

    database_path = pwd[:-6] + "flightData/filtered_data/test_set/scenario"+str(scenario_num)
    database_file_names = []
    for root, dirs, files in os.walk(database_path):
        database_file_names = [os.path.join(root,file) for file in files]
    scores = []
    for i,file in enumerate(database_file_names):
        data = pd.read_csv(file)
        X_test = data.drop(columns=['label', 'time','x','y','pitch_v','roll_v','yaw_v'])
        X_test = X_test.values
        Y_test = data['label'].values
        Y_test = Y_test.reshape((-1,1))

        X_test = scaler.transform(X_test)
        Y_test = encoder.transform(Y_test).toarray()

        input_data = X_test[:-time_step,:]
        targets = Y_test[time_step:,:]
        dataset = timeseries_dataset_from_array(input_data, 
                                                targets, 
                                                sequence_length=time_step,
                                                batch_size=128)

        y_pred = model.predict(dataset,
                              verbose=0,
                              callbacks=None,#[batch_print_callback],
                              max_queue_size=512,
                              workers=80,
                              use_multiprocessing=True)

        y_pred = encoder.inverse_transform(y_pred)
        y_true = encoder.inverse_transform(targets[0:y_pred.shape[0],:])
        
        report = classification_report(y_true,y_pred,output_dict=True)
        scores.append([report['accuracy'],report['weighted avg']['precision'],
                       report['weighted avg']['recall'],report['weighted avg']['f1-score']])


    scores = np.array(scores)
    test_result['Scenaario-'+str(scenario_num)] = np.mean(scores,axis=0)
test_result = pd.DataFrame(test_result,index=['Accuracy','Precision','Recall','F1_Score'])
print(test_result)

           Scenaario-1  Scenaario-2  Scenaario-3  Scenaario-4  Scenaario-5
Accuracy      0.962353     0.930642     0.940264     0.889389     0.842161
Precision     0.964333     0.981028     0.942255     0.981423     0.865621
Recall        0.962353     0.930642     0.940264     0.889389     0.842161
F1_Score      0.959941     0.951812     0.939302     0.930916     0.847543
CPU times: user 5min 10s, sys: 1min 14s, total: 6min 25s
Wall time: 1min 23s


# Decision tree Implementation

# Training

In [54]:
%%time
clf = tree.DecisionTreeClassifier(criterion='gini',#'entropy',#'gini', 
                                              splitter='best', 
                                              max_depth=None,#default None 
                                              min_samples_split=2, 
                                              min_samples_leaf=1, 
                                              min_weight_fraction_leaf=0.0, 
                                              max_features=None, 
                                              random_state=None, 
                                              max_leaf_nodes=None, 
                                              min_impurity_decrease=0.0, 
                                              min_impurity_split=None, 
                                              class_weight=None, 
                                              presort=False)

#train decision tree
pwd = os.getcwd()
database_path = pwd[:-6] + "flightData/filtered_data/scenario5"
database_file_names = []
for root, dirs, files in os.walk(database_path):
    database_file_names = [os.path.join(root,file) for file in files]

scaler = MinMaxScaler()
train_data = None
train_files = database_file_names[:-1]
test_files = [database_file_names[-1]]
X_train_all = None
Y_train_all = None
for i,file in enumerate(train_files):
    data = pd.read_csv(file)
    data = data.replace({'Hold':1, 'Takeoff':2, 'Hover':3, 'Search':4, 'Obstacleavoid':5, 'Loiter':6, 'Land':7})
    X_train = data.drop(columns=['label', 'time','x','y','pitch_v','roll_v','yaw_v'])
    X_train = X_train.values
    Y_train = data['label'].values
    Y_train = Y_train.reshape((-1,1))
    if i == 0:
        X_train = scaler.fit_transform(X_train)
    else:
        X_train = scaler.transform(X_train)

    input_data = X_train[:-time_step,:]
    targets = Y_train[time_step:,:]
    dataset = timeseries_dataset_from_array(input_data, 
                                            targets, 
                                            sequence_length=time_step,
                                            batch_size=128)
    for x,y in dataset:
        x = x.numpy()
        y = y.numpy()
        if X_train_all is None:
            X_train_all = x.reshape((-1,time_step*7))
            Y_train_all = y
        else:
            X_train_all = np.append(X_train_all, x.reshape((-1,time_step*7)), axis=0)
            Y_train_all = np.append(Y_train_all, y, axis=0)
        
        
# X_train = train_data.drop(columns=['label', 'time','x','y','pitch_v','roll_v','yaw_v'])
# X_train = X_train.values
# Y_train = train_data['label'].values
# Y_train = Y_train.reshape((-1,1))


clf = clf.fit(X_train_all, Y_train_all)

CPU times: user 11min 29s, sys: 6min 10s, total: 17min 39s
Wall time: 15min 50s


# Validation

In [55]:
%%time
X_test_all = None
Y_test_all = None
for i,file in enumerate(test_files):
    data = pd.read_csv(file)
    data = data.replace({'Hold':1, 'Takeoff':2, 'Hover':3, 'Search':4, 'Obstacleavoid':5, 'Loiter':6, 'Land':7})
    X_test = data.drop(columns=['label', 'time','x','y','pitch_v','roll_v','yaw_v'])
    X_test = X_test.values
    Y_test = data['label'].values
    Y_test = Y_test.reshape((-1,1))
    
    X_test = scaler.transform(X_test)

    input_data = X_test[:-time_step,:]
    targets = Y_test[time_step:,:]
    dataset = timeseries_dataset_from_array(input_data, 
                                            targets, 
                                            sequence_length=time_step,
                                            batch_size=128)
    for x,y in dataset:
        x = x.numpy()
        y = y.numpy()
        if X_test_all is None:
            X_test_all = x.reshape((-1,time_step*7))
            Y_test_all = y
        else:
            X_test_all = np.append(X_test_all, x.reshape((-1,time_step*7)), axis=0)
            Y_test_all = np.append(Y_test_all, y, axis=0)
y_pred = clf.predict(X_test_all) 

print(accuracy_score(Y_test_all, y_pred))

0.9013649190839108
CPU times: user 31.5 s, sys: 32.4 s, total: 1min 3s
Wall time: 14.4 s


# Testing

In [56]:
%%time
# testing
pwd = os.getcwd()

test_result = {}

for scenario_num in range(1,6):

    database_path = pwd[:-6] + "flightData/filtered_data/test_set/scenario"+str(scenario_num)
    database_file_names = []
    for root, dirs, files in os.walk(database_path):
        database_file_names = [os.path.join(root,file) for file in files]
    scores = []
    X_test_all = None
    Y_test_all = None
    for i,file in enumerate(database_file_names):
        data = pd.read_csv(file)
        data = data.replace({'Hold':1, 'Takeoff':2, 'Hover':3, 
                             'Search':4, 'Obstacleavoid':5, 'Loiter':6, 'Land':7})
        X_test = data.drop(columns=['label', 'time','x','y','pitch_v','roll_v','yaw_v'])
        X_test = X_test.values
        Y_test = data['label'].values
        Y_test = Y_test.reshape((-1,1))

        X_test = scaler.transform(X_test)
        

        input_data = X_test[:-time_step,:]
        targets = Y_test[time_step:,:]
        dataset = timeseries_dataset_from_array(input_data, 
                                                targets, 
                                                sequence_length=time_step,
                                                batch_size=128)
        for x,y in dataset:
            x = x.numpy()
            y = y.numpy()
            if X_test_all is None:
                X_test_all = x.reshape((-1,time_step*7))
                Y_test_all = y
            else:
                X_test_all = np.append(X_test_all, x.reshape((-1,time_step*7)), axis=0)
                Y_test_all = np.append(Y_test_all, y, axis=0)

    y_pred = clf.predict(X_test_all)
        
    report = classification_report(Y_test_all,y_pred,output_dict=True)
    scores = [report['accuracy'],report['weighted avg']['precision'],report['weighted avg']['recall'],report['weighted avg']['f1-score']]

    test_result['Scenaario-'+str(scenario_num)] = scores
test_result = pd.DataFrame(test_result, index=['Accuracy','Precision','Recall','F1_Score'])
print(test_result)

           Scenaario-1  Scenaario-2  Scenaario-3  Scenaario-4  Scenaario-5
Accuracy      0.849008     0.918647     0.926204     0.897765     0.901365
Precision     0.921246     0.954605     0.932536     0.977313     0.911999
Recall        0.849008     0.918647     0.926204     0.897765     0.901365
F1_Score      0.860320     0.935031     0.929184     0.935396     0.903719
CPU times: user 15min 56s, sys: 23min 53s, total: 39min 50s
Wall time: 34min 31s
